In [91]:
import pyodbc
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime, timedelta

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from googletrans import Translator

from transform_copy import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eats\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [93]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [94]:
tables = engine.table_names()

C:\Users\eats\AppData\Local\Temp\ipykernel_21692\863585613.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


In [121]:
df_job = pd.DataFrame(engine.execute(
            """
            SELECT Job.JobID, Job.JobTitle, FunctionPosition.FunctionPositionName, EducationLevel.EducationLevelName, City.Name AS CityName, Province.Name AS ProvinceName, Major.MajorName, Job.Description, Job.Requirement
            FROM (((((Job
            RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
            RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
            RIGHT JOIN City ON Job.CityID = City.CityID)
            RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
            RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
            WHERE JobStatus='Publish'
            """
        ))

In [122]:
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

df_job = df_job.applymap(str.lower)
df_job.EducationLevelName = df_job.EducationLevelName.replace('none', '')

translator = Translator(service_urls=['translate.googleapis.com'])

df_job.JobTitle = df_job.JobTitle.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.FunctionPositionName = df_job.FunctionPositionName.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.MajorName = df_job.MajorName.apply(lambda x: translator.translate(x, dest='id').text.lower())

df_job.Description = df_job.Description.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.Requirement = df_job.Requirement.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())

In [124]:
df_job['JobTitle'] = df_job.JobTitle.str.cat(
    df_job.FunctionPositionName.str.cat(
        df_job.MajorName
    , sep=' ')
, sep=' ').apply(lambda x: ' '.join(set(x.split(' ')))).map(remove_standalonesymbols).map(remove_parenthesesnumber).map(remove_morespace)

df_job.drop(columns=['FunctionPositionName', 'MajorName'], inplace=True)

In [101]:
for i,j in df_job[['Translated', 'JobTitle']].values:
    print(i, '|', j)

direksi sekretaris | direksi sekretaris
staf desain grafis | design desain staff graphic grafis
toko jurusan semua staf awak kapal | jurusan staff crew semua outlet
toko jurusan sma segala awak kapal Pengelola | jurusan sma crew segala outlet manager
pengawas akuntansi | accounting supervisor akuntansi
hukum Pengelola | hukum manager legal
umum qc lapangan sipil operasi teknik | general qc operation lapangan sipil teknik
pajak pengawas akuntansi | pajak accounting supervisor akuntansi
riset jurusan semua staf perkembangan bisnis | research jurusan staff semua business & development
umum jurusan perselingkuhan semua staf | general jurusan staff semua affair
keuangan staf akuntansi  | staff akuntansi & accounting finance
montir listrik jurusan bm-saya semua spv pemeliharaan | bm-me maintenance jurusan semua spv electician 2
Properti jurusan semua pemasaran mendukung | support property jurusan semua marketing
staf informatika dia teknik | informatika staff it teknik
direksi sekretaris sem

In [97]:
df_job.isna().sum()

JobTitle              0
EducationLevelName    0
CityName              0
ProvinceName          0
Description           0
Requirement           0
Translated            0
dtype: int64